In [1]:
import random

import pandas as pd

import torch
import torch.nn            as nn
import torch.nn.functional as F
import torch.optim         as optim
from torch.nn.utils import weight_norm
from torch.autograd import Variable

In [2]:
cuda          = True
file_path     = '../data/horoscope_2011.csv'
window_size   = 120
batch_size    = 256
print_every   = 400
test_seq_size = 500
epochs        = 20

In [3]:
def load_data(path, window_size):
    df               = pd.read_csv(path)
    text             = ' '.join(df.TEXT)
    characters       = set(text)
    n_characters     = len(characters)
    idx_to_character = dict(enumerate(characters))
    character_to_idx = {character : idx for idx, character in idx_to_character.items()}
    data             = [(text[i : i + window_size], text[i + 1 : i + window_size + 1])
                        for i in range(len(text) - window_size - 1)]
    
    return n_characters, idx_to_character, character_to_idx, data

In [4]:
def one_hot_encode(inp_tensor, length):
    inp_                = torch.unsqueeze(inp_tensor, 2)
    batch_size, seq_len = inp_tensor.size()
    one_hot             = torch.FloatTensor(batch_size, seq_len, length).zero_()
    one_hot.scatter_(2, inp_, 1).transpose_(1, 2)
    
    return one_hot

In [5]:
def encode_seq(seq, char_to_id):
    return [char_to_id[c] for c in seq]

In [6]:
def data_to_tensor(data, char_to_id, n_char):
    input_tensor    = torch.LongTensor([encode_seq(input_seq, char_to_id) 
                                         for input_seq, _ in data])
    one_hot_input   = one_hot_encode(input_tensor, n_char)
    target_tensor   = torch.LongTensor([encode_seq(target_seq, char_to_id) 
                                         for _, target_seq in data])

    return one_hot_input, target_tensor

In [7]:
def batch_generator(data, batch_size, n_char, char_to_id, shuffle = True):
    if shuffle:
        data = random.sample(data, len(data))
    
    return (data_to_tensor(data[i : i + batch_size], char_to_id, n_char) 
                 for i in range(0, len(data), batch_size))

In [8]:
def test_model(tcn, final_sequence_size, window_size, n_char, id_to_char, 
               char_to_id, data):
    seq = list(random.choice(data)[0])
    while len(seq) < final_sequence_size:
        encoded_input = encode_seq(seq[-window_size:], char_to_id)
        input_tensor  = torch.LongTensor([encoded_input])
        one_hot_input = one_hot_encode(input_tensor, n_char)
        X             = Variable(one_hot_input)
        X             = X.cuda() if cuda else X 
        y_pred        = tcn(X)
        char_pred_id  = y_pred[0].max(dim = -2)[1][-1].cpu().data[0]
        char_pred     = id_to_char[char_pred_id]
        seq.append(char_pred)
    
    return ''.join(seq)

In [9]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size
        
    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

In [10]:
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout = 0.2):
        super(TemporalBlock, self).__init__()
        conv_params = {
            'kernel_size' : kernel_size,
            'stride'      : stride,
            'padding'     : padding,
            'dilation'    : dilation
        }
        self.conv1    = weight_norm(nn.Conv1d(n_inputs, n_outputs, **conv_params))
        self.chomp1   = Chomp1d(padding)
        self.relu1    = nn.ReLU()
        self.dropout1 = nn.Dropout2d(dropout)
        self.conv2    = weight_norm(nn.Conv1d(n_outputs, n_outputs, **conv_params))
        self.chomp2   = Chomp1d(padding)
        self.relu2    = nn.ReLU()
        self.dropout2 = nn.Dropout2d(dropout)
        self.net      = nn.Sequential(
            self.conv1, 
            self.chomp1,
            self.relu1,
            self.dropout1,
            self.conv2,
            self.chomp2,
            self.relu2,
            self.dropout2
        )
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu       = nn.ReLU()
        self.init_weights()
        
    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)

        return self.relu(out + res)
    
    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

In [11]:
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size = 2, dropout = 0.2):
        super(TemporalConvNet, self).__init__()
        layers     = []
        num_levels = len(num_channels)
        
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels   = num_inputs if i == 0 else num_channels[i - 1]
            out_channels  = num_channels[i]
            layers.append(
                TemporalBlock(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride   = 1,
                    dilation = dilation_size,
                    padding  = (kernel_size - 1) * dilation_size,
                    dropout  = dropout
                )
            )
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)

In [12]:
n_char, id_to_char, char_to_id, data = load_data(file_path, window_size)
num_channels                         = [32] * 6 + [n_char]
tcn                                  = TemporalConvNet(n_char, num_channels)
tcn                                  = tcn.cuda() if cuda else tcn
criterion                            = nn.CrossEntropyLoss()
optimizer                            = optim.Adam(tcn.parameters())

In [13]:
for epoch in range(epochs):
    running_loss = 0
    for i, (X, y) in enumerate(batch_generator(data, batch_size, n_char, char_to_id)):
        X = Variable(X)
        y = Variable(y)
        X = X.cuda() if cuda else X
        y = y.cuda() if cuda else y
        optimizer.zero_grad()
        y_pred = tcn(X)
        loss   = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.cpu().data[0]
        
        if i % print_every == print_every - 1:
            print(f'[{epoch + 1 : 3}|{i + 1 : 6}] loss: {running_loss / print_every:.4f}')
            test_result = test_model(tcn, test_seq_size, window_size, n_char, id_to_char, char_to_id, data)
            print(f'\n{test_result}\n')
            running_loss = 0

[  1|   400] loss: 2.8097

work, yet you might talk about everything you want to do instead of actually doing it today. You are motivated by a stron to to your the to to the to the to to to to to to to the the to to the to to to the to to the to to the to the to to to to the to to the the te the to the the to to the to to to the the tere to teng the the ten the the teter to the the to ther the to the the to tour the to to the the ther the to to the to to the to the the the the to to to the to to to the to to to to the to 

[  1|   800] loss: 2.1799

ll lead. You're typically a better team player than a solo act, yet you're still motivated to prove that you're capable to than the to the to the tore to the the the the the the the the to the the the to ther you to ther the the the the the the the to the the the the the the the the to the the the to the the to the to the the the the the to the the to the the to the to the the the to the the the the the the the to the the the the the the th

[  1|  6800] loss: 1.6277

e thinking about improving your daily regimen, but could get sidetracked by worrying about the impact your lifestyle change your for your to staning your are to and to the proble to the sone to a to an a the of to your some the preacion to a something to the prort to the some the ontime that you may trul the be the sone to seem to your the strational be a self of your alle the sore the the something the beca the stretence to stans to a someting to sore to the a some the some the stound of your s

[  1|  7200] loss: 1.6234

citing place is in front of you, so don't ruin it by starting without taking the time to consider the consequences of your stration the someone to you make your feelings to be a something your stout the strations tout you are some the presting the really the someone a may to the mation to a see to the so action to sould be the strations the st the probest to a strate the some that you to still to a mome to and to stound that you are an a st

[  2|  6000] loss: 1.5814

s as today's New Moon in your 6th House of Work reveals an increasing need to be practical. Making something original is to and the see the probest to the prouge to and the the feel to the the the Moon to your the Moon's be the prest the that you can the some the ant to someon the Moon in your the situation to are the the of to your prore to stanting to strate the see the pron to a see the the stil to and the problead to start to your sementing to something to a something to strate the sect to a

[  2|  6400] loss: 1.5802

tead of trying to explain everything you want to do, just tell everyone that you need a couple of days to sort yourself the prort to stant the still the stand the streng the the prest to your the somet that you to so be and to action to a see the stant the stand to and to so a something to and that you are the someone the that you that you may be a seek the that you are and that you could be the the Moon inder to be and to support to the st

[  3|  5200] loss: 1.5649

an the simple truth. Unfortunately, your desire to know another person's feelings could drive your own emotions deeper in your prealing to and the pressions the see the stiot to the somether to your plans to a send to a contilly the promether to some than you could be the presser to a to a could the to stand to a strate a set to settens that your prealions to a so the pow the still the the the be a strere to a stantion to your to your from to stant the do and to s seek the to strate thought to s

[  3|  5600] loss: 1.5629

n below the radar today. If your co-workers are aware of your positive energy, they will likely expect even more from your and to strate to are the pressing the to some there the press the some to some the stould be and and be the strenge the proble to the be the sthe the may to see the Moon in the stand to a see the may the the presintain to your and stant the start to your plans to a seem to strations the see the some the because your pro

[  4|  4400] loss: 1.5565

you won't likely be so judgmental later in the week. It's no fun being around a controlling person, yet you might be the from the someor the strate to a something to so the be and the pressing to the deations the fres to a probable to a be a to stead and the deally to stand to a someone the stant to the start to stand to something your profesintions your stake and to your strenging to seem to the Moon in your because your proation to your actions to the seem the probles to the to a come to atten

[  4|  4800] loss: 1.5564

ossibilities. Instead, pick one goal, write it down and then commit to do whatever it takes to make your dreams come trution to see that you can a may be a to strate to steant to your alsting to a could be the proges to sition to the some the to seek to and the sest to see the proble to a may be the fround the stout the and to the sthere the surten to the stre to stop to make the the and to sting to the some the the strate the prout the sta

...

[ 17|   400] loss: 1.5329

l and professional wheels. The time for thinking is over; get to work and apply your wealth of knowledge to getting ahead to someone the may to and the the stide tome to still something to sine that you may be a consilf to so and to be a be a to the stant to may be a sent to strong the proble to and to and to strate the strate to your sime to the be a strateg the the the prose the oftintion to and to the stay to so your the present to see the see you may be a stant your conting to somet your som

[ 17|   800] loss: 1.5329

hardest jobs out of the way at the beginning of the day. Lucky Jupiter harmonizes with Venus in your 2nd House of Money the from to the proble to a confider you may be a may be a consent the take the some to your could be and the strateger, you the stant to the promestic to that you may be the some the persone to a so the stay to a by the stick to and the the see your planet to be a strate to a making the stret to someone that you may be an

[ 17|  6800] loss: 1.5322

 caution doesn't mean that you are uncertain. You just want more time for everything to settle down so you can decide homeone you may be and to still and to a so be able to and that you could be a to the the process to your and to the suncery the may be a to a to the to stant to are about the comple to the the the the doth to and to a some the that you might to the making the some the the the the dest you make to so the to be about the the presting to and to stible to there to some your probld t

[ 17|  7200] loss: 1.5327

ively innocent question. Take a deep breath and think twice before opening your mouth. An unforeseeable event could tempt to make to and the that you are the streter to may the make a for the a concess to your feel antill so you may be the stang to start the still to the prodess to your from and the day be the Moon in your feel to and the car the that you the that you may be and the make and to a to and the still so the strate to a to a str

[ 18|  6000] loss: 1.5306

 plane tickets. Your positive attitude goes a long way on your road to success, so if you're seeking adventure, there's to stions to the seem to a consider to the becantion to a someone the sore to and to a to your actionsh your perient the to sting to sthe for your feelings to a more to strong to the to a so the make the progrsing to the that you to and the because the strate to the to and the of the don't the dest to your seem to the somethind to a for the streat to at you to so stions to a se

[ 18|  6400] loss: 1.5319

hat requires hard work. Keep in mind that you'll feel better once you have something physical to do. The complexity of your pron your the start to some to the Moon intense the a confically the the stro to the stay to seem the stratege to the the the prow you to something the feel to the Moon in your feeling the to a some to and to a for the the probant that you coge the stractic to stick to your into that you to stant you can that you may b

[ 19|  5200] loss: 1.5313

tes your extra effort. Sometimes taking care of everyone else can leave you exhausted. Don't rely on getting approval for that you set your from the stand to the may be the somether to be a beenge your for to the see you may be a may be and to may be a make to seem to be the to may be to and that you to be and the strations the may be a seem the better to so your ant the someone the something to a might your feelings your feel to be the Moon and the seed to the strate the day to a stay the probl

[ 19|  5600] loss: 1.5311

 hearing conflicting stories. However, you're not very receptive because you are afraid of being misled by someone else's a stand to so a to may be the strate to the disering to and to make to a confintenge to the probld to still the promuch that you may the make a some the stake to your feel and to may be and the strerent your perspective the conce the Moon in your for the start to make the feed to a someone are and that they and the strat

[ 20|  4400] loss: 1.5292

raight and narrow path, give yourself permission to be inspired by your imagination. Don't try to justify your choices to your a don't the may be the probling your press to stang to and the that you cant to be an your from to and to the stration to becan the sen the some to your strategy the pressing to and the to a so the despect to the still your at your fer to the cay the confress to the stand to and that you concer a sting of your and and to and to a could be and to a may be the Moone to sta

[ 20|  4800] loss: 1.5297

y and you aren't able to get to the bottom of an issue because your judgment isn't as sharp as usual. Don't push too har the don't be the the prow the probabous that you to stre to the to make to the stare the the prover the spead to the strong the prow you to start to strations to the strate to and to a to your antitions the prark that yout and that you to a so you may be the start to your stant to the something the someone the Moon into y